# Spectral transformations

There are several spectral transformation methods in Earth Engine. These include instance methods on images such as `normalizedDifference()`, `unmix()`, `rgbToHsv()` and `hsvToRgb()`. The latter two methods are useful for pan sharpening. For example:

#### Earth Engine setup

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

#### Folium setup (for interactive map display)

In [ ]:
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

#### Begin examples

In [ ]:
# Load a Landsat 8 top-of-atmosphere reflectance image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Convert the RGB bands to the HSV color space.
hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()

# Swap in the panchromatic band and convert back to RGB.
sharpened = ee.Image.cat(
    [hsv.select('hue'),
     hsv.select('saturation'),
     image.select('B8')]).hsvToRgb()

# Define a map centered on San Francisco, California.
map_sharpened = folium.Map(location=[37.76664, -122.44829], zoom_start=13)

# Add the image layers to the map and display it.
map_sharpened.add_ee_layer(image, {
    'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]
}, 'rgb')
map_sharpened.add_ee_layer(sharpened, {
    'min': 0, 'max': 0.25, 'gamma': [1.3, 1.3, 1.3]
}, 'pan-sharpened')
display(map_sharpened.add_child(folium.LayerControl()))

Spectral unmixing is implemented in Earth Engine as the `image.unmix()` method. (For more flexible methods, see the [Array Transformations page](https://developers.google.com/earth-engine/guides/arrays_transformations)). The following is an example of unmixing Landsat 5 with predetermined urban, vegetation and water endmembers:

In [ ]:
# Load a Landsat 5 image and select the bands we want to unmix.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
image = ee.Image('LANDSAT/LT05/C01/T1/LT05_044034_20080214').select(bands)

# Define spectral endmembers.
urban = [88, 42, 48, 38, 86, 115, 59]
veg = [50, 21, 20, 35, 50, 110, 23]
water = [51, 20, 14, 9, 7, 116, 4]

# Unmix the image.
fractions = image.unmix([urban, veg, water])

# Define a map centered on San Francisco Bay.
map_fractions = folium.Map(location=[37.5010, -122.1899], zoom_start=10)

# Add the image layers to the map and display it.
map_fractions.add_ee_layer(
    image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 128}, 'image')
map_fractions.add_ee_layer(fractions, None, 'unmixed')
display(map_fractions.add_child(folium.LayerControl()))

The unmixing result should look something like Figure 1.

![unmixed_sf](https://developers.google.com/earth-engine/images/Images_unmixing_sf.png) Figure 1. Landsat 5 imagery unmixed to urban (red), vegetation (green) and water (blue) fractions. San Francisco bay area, California, USA.